<a href="https://colab.research.google.com/github/gforconi/UTNIA2025/blob/main/NLP_Conceptos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Clase de NLP — Demo práctica
**Contenidos:** Tokenización y lematización (NLTK), clasificación de sentimientos con TF‑IDF + Naive Bayes (scikit‑learn), **mini‑RAG** (búsqueda + generación simple) y **agente** de juguete con herramientas.

> Ejecutá las celdas en orden. Si es la primera vez, corré la celda de *instalación rápida*.


## 🚀 Instalación rápida (si te falta algo)

**NLTK (Natural Language Toolkit)**

Es una librería de Python para Procesamiento de Lenguaje Natural (PLN).

Proporciona herramientas para:

- Tokenizar (separar palabras y oraciones).

- Eliminar stopwords (palabras sin valor semántico como “el”, “de”).

- Stemming y lematización (reducir palabras a su raíz).

- Trabajar con corpus (ejemplos) de texto.

Se usa mucho en investigación, enseñanza y prototipos de análisis de texto.

**scikit-learn**

Permite:

- Clasificación, regresión, clustering.

- Preprocesamiento de datos (vectorización de texto, escalado, normalización).

- Modelos estadísticos y algoritmos de ML (Naive Bayes, SVM, árboles, etc.).

- Muy usada para crear y entrenar modelos predictivos de forma rápida y sencilla.

In [3]:

# Ejecutá esta celda si no tenés las librerías instaladas.
# (Podés volver a ejecutarla sin problemas)
import sys, subprocess

def pip_install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", pkg])

for pkg in ["nltk", "scikit-learn"]:
    try:
        __import__(pkg.split("==")[0])
    except Exception:
        pip_install(pkg)

import nltk
# Descargar recursos necesarios para tokenización y lematización
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab')
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

print("Listo ✅")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Listo ✅


## 1) Tokenización y lematización (NLTK)

**Tokenizar:** Es el proceso de dividir un texto en unidades más pequeñas llamadas tokens.

Un token puede ser una palabra, un número, un signo de puntuación o incluso una oración, según cómo lo definas.

Se pueden tokenizar palabras, oraciones, parrafos.

**Lematizar:** Es reducir una palabra a su forma base o “lema” (la que encontrarías en un diccionario). Tiene en cuenta la gramática y el contexto.

**Stemming:** es una técnica que recorta las palabras a su raíz (stem), sin importar si la raíz es una palabra válida en el idioma. Se basa en reglas simples de cortar sufijos y prefijos.

El stem puede no ser una palabra válida, pero sirve para agrupar variantes parecidas.

**Lematizar vs Stemming:**

Lematizar es más avanzado que un stemming, ya que el stemming solo corta palabras).

Stemming es más mecánico y agresivo que la lematización.


In [4]:

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

texto = "Los modelos de lenguaje grandes están transformando el NLP rápidamente."
tokens = word_tokenize(texto.lower(), language='spanish')
lemmatizer = WordNetLemmatizer()
lemmas = [lemmatizer.lemmatize(t) for t in tokens]

print("Tokens:", tokens)
print("Lemmas:", lemmas)


Tokens: ['los', 'modelos', 'de', 'lenguaje', 'grandes', 'están', 'transformando', 'el', 'nlp', 'rápidamente', '.']
Lemmas: ['los', 'modelos', 'de', 'lenguaje', 'grandes', 'están', 'transformando', 'el', 'nlp', 'rápidamente', '.']


In [6]:
text = "The cats are running faster than the dogs"

# Tokenización
tokens = word_tokenize(text)
print("Tokens:", tokens)

# Lematización
lemmatizer = WordNetLemmatizer()
lemmas = [lemmatizer.lemmatize(t) for t in tokens]
print("Lemmas:", lemmas)

Tokens: ['The', 'cats', 'are', 'running', 'faster', 'than', 'the', 'dogs']
Lemmas: ['The', 'cat', 'are', 'running', 'faster', 'than', 'the', 'dog']


"cats" → "cat"

"dogs" → "dog"

"running" no cambia porque la lematización de NLTK necesita la parte de la oración (POS tag) para hacerlo bien.

## 2) Clasificación de sentimientos con TF‑IDF + Naive Bayes

Las principales librerias que vamos a utilizar son:

**TfidfVectorizer:** transforma el texto en vectores numéricos usando la técnica TF-IDF (Term Frequency – Inverse Document Frequency).
→ Sirve para representar qué tan importantes son las palabras en cada oración.

**MultinomialNB:** clasificador Naive Bayes Multinomial, muy usado para texto porque funciona bien con conteos/frecuencias de palabras.

**make_pipeline:** crea un pipeline que encadena varias transformaciones y un modelo en una sola estructura.

In [1]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

# Mini dataset de ejemplo (español)
# Frases
X = [
    "Me encanta este producto, funciona de maravilla",
    "Horrible experiencia, no lo recomiendo a nadie",
    "Excelente atención al cliente, muy satisfecho",
    "Malo y defectuoso, perdí mi dinero",
    "Buen desempeño y calidad aceptable",
    "Terrible, se rompe a los dos días",
    "Fantástico, superó mis expectativas",
    "Pésimo servicio y respuesta lenta"
]
# Etiqueta de cada frase
y = ["pos", "neg", "pos", "neg", "pos", "neg", "pos", "neg"]

# Creacion del modelo:
# Aquí el pipeline hace dos cosas:
# 1. TfidfVectorizer
#     ngram_range=(1,2) → usa tanto palabras individuales (unigramas) como pares de palabras consecutivas (bigramas).
#     min_df=1 → incluye términos que aparecen al menos en 1 documento.
# 2. MultinomialNB → clasifica el texto transformado en TF-IDF como pos o neg.

modelo = make_pipeline(TfidfVectorizer(ngram_range=(1,2), min_df=1), MultinomialNB())

# Entrenamiento
modelo.fit(X, y)

pruebas = ["muy bueno y recomendable", "esto es una estafa", "calidad normal", "no sirve"]
pred = modelo.predict(pruebas)

for t, p in zip(pruebas, pred):
    print(f"{t!r} -> {p}")


'muy bueno y recomendable' -> pos
'esto es una estafa' -> neg
'calidad normal' -> pos
'no sirve' -> neg


En resumen:

Este script es un clasificador de sentimientos muy básico en español.

- Usa TF-IDF para convertir texto en números.

- Usa Naive Bayes para clasificar en positivo o negativo.

- Demuestra cómo entrenar y luego usar el modelo para predecir frases nuevas.

### Mirar características TF‑IDF

In [3]:
vect = modelo.named_steps['tfidfvectorizer']
nb = modelo.named_steps['multinomialnb']

feature_names = vect.get_feature_names_out()
import numpy as np

# Top características más influyentes para 'pos' y 'neg'
class_idx_pos = list(nb.classes_).index('pos')
top_pos = np.argsort(nb.feature_log_prob_[class_idx_pos])[-10:]
class_idx_neg = list(nb.classes_).index('neg')
top_neg = np.argsort(nb.feature_log_prob_[class_idx_neg])[-10:]

print("Top POS:", feature_names[top_pos])
print("Top NEG:", feature_names[top_neg])

Top POS: ['buen desempeño' 'calidad' 'desempeño calidad' 'expectativas' 'aceptable'
 'buen' 'desempeño' 'calidad aceptable' 'superó mis' 'superó']
Top NEG: ['mi dinero' 'perdí mi' 'perdí' 'pésimo servicio' 'pésimo' 'lenta'
 'respuesta' 'servicio respuesta' 'respuesta lenta' 'servicio']


## 3) Mini‑RAG (búsqueda + generación simple)


**Idea:** guardamos documentos cortos, buscamos los más relevantes con TF‑IDF y **generamos** una respuesta tomando frases de esos documentos.  
> No requiere API externas ni claves.


In [4]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

docs = [
    "El NLP permite que las computadoras entiendan y generen lenguaje humano. Involucra tareas como clasificación, NER y traducción.",
    "RAG combina recuperación de información con modelos generativos. Primero busca pasajes relevantes y luego el LLM genera la respuesta.",
    "Los embeddings o TF-IDF pueden representar documentos para búsqueda semántica. Luego se seleccionan los top-k más similares.",
    "Un agente puede decidir qué herramienta usar (calculadora, búsqueda, API) antes de generar una respuesta final."
]

doc_ids = [f"D{i+1}" for i in range(len(docs))]

# Indexado
vectorizer = TfidfVectorizer(stop_words=None)
M = vectorizer.fit_transform(docs)

def retrieve(query, k=2):
    qv = vectorizer.transform([query])
    sims = cosine_similarity(qv, M).ravel()
    idx = sims.argsort()[::-1][:k]
    return [(doc_ids[i], docs[i], float(sims[i])) for i in idx]

def simple_generate(query, retrieved):
    # Generación muy simple: concatena oraciones relevantes y agrega un cierre.
    context = " ".join([r[1] for r in retrieved])
    # Selección de frases que contienen palabras clave
    import re
    sentences = re.split(r'(?<=[.!?])\s+', context)
    keywords = [w for w in query.lower().split() if len(w) > 3]
    chosen = [s for s in sentences if any(k in s.lower() for k in keywords)]
    if not chosen:
        chosen = sentences[:2]
    answer = " ".join(chosen)
    return answer + " (Respuesta generada a partir de pasajes recuperados)."

query = "¿Qué es RAG y cómo se usa con un LLM?"
retrieved = retrieve(query, k=2)
print("Pasajes recuperados:")
for rid, text, score in retrieved:
    print(f"- {rid} (score={score:.3f}): {text}")

print("\nRespuesta:")
print(simple_generate(query, retrieved))


Pasajes recuperados:
- D2 (score=0.302): RAG combina recuperación de información con modelos generativos. Primero busca pasajes relevantes y luego el LLM genera la respuesta.
- D4 (score=0.212): Un agente puede decidir qué herramienta usar (calculadora, búsqueda, API) antes de generar una respuesta final.

Respuesta:
RAG combina recuperación de información con modelos generativos. Primero busca pasajes relevantes y luego el LLM genera la respuesta. (Respuesta generada a partir de pasajes recuperados).


## 4) Agente de juguete (herramientas + traza)


**Flujo:** El "agente" decide si usar la **calculadora** o la **búsqueda** (mini‑RAG) según la consulta.  
Mostramos una **traza** de decisiones para explicar qué hizo.


In [5]:

import operator
import re

def tool_calculator(expr: str):
    # Seguridad básica: solo números, espacio y operadores + - * / ( )
    if not re.fullmatch(r"[0-9+\-*/().\s]+", expr):
        raise ValueError("Expresión no permitida")
    return eval(expr)

def tool_search(query: str):
    return retrieve(query, k=2)

def agent(query: str):
    trace = []
    # Decidir herramienta
    if re.search(r"[0-9]\s*[+\-*/]", query) or any(w in query.lower() for w in ["calcula", "sumá", "multiplicá"]):
        trace.append("Decisión: usar calculadora")
        expr = re.findall(r"[0-9+\-*/().\s]+", query)
        expr = expr[0] if expr else query
        result = tool_calculator(expr)
        trace.append(f"Calculadora -> {expr} = {result}")
        answer = f"El resultado es {result}."
    else:
        trace.append("Decisión: usar búsqueda (mini‑RAG)")
        hits = tool_search(query)
        trace.append(f"Recuperados: {[h[0] for h in hits]}")
        answer = simple_generate(query, hits)
    return trace, answer

# Pruebas
for q in ["Calcula 12 * (3 + 4)", "¿Qué es un agente en NLP?" ]:
    t, a = agent(q)
    print("Consulta:", q)
    print("Traza:")
    for step in t:
        print(" -", step)
    print("Respuesta:", a)
    print("-"*60)


Consulta: Calcula 12 * (3 + 4)
Traza:
 - Decisión: usar calculadora
 - Calculadora ->  12 * (3 + 4) = 84
Respuesta: El resultado es 84.
------------------------------------------------------------
Consulta: ¿Qué es un agente en NLP?
Traza:
 - Decisión: usar búsqueda (mini‑RAG)
 - Recuperados: ['D4', 'D1']
Respuesta: Un agente puede decidir qué herramienta usar (calculadora, búsqueda, API) antes de generar una respuesta final. (Respuesta generada a partir de pasajes recuperados).
------------------------------------------------------------



---
### ✅ ¿Qué aprendimos?
- Cómo **preprocesar** texto (tokenizar/lematizar).
- Entrenar un **clasificador** simple con TF‑IDF + Naive Bayes.
- Armar un **mini‑RAG** sin dependencias externas.
- Implementar un **agente** de juguete que decide herramientas y expone su traza.

> Para llevarlo al siguiente nivel, vamos a cambiar TF‑IDF por **embeddings** y conectar un LLM real. Ayudados por el framework [LangChain](https://www.langchain.com/)
